In [75]:
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torch.autograd import Variable
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import os
import cv2
from PIL import Image
from tqdm import tqdm_notebook as tqdm
import random
from matplotlib import pyplot as plt
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
import csv
import pandas as pd
import numpy as np

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [76]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [77]:
def model1(x):
    PATH = '/Users/parkjeonghoon/Desktop/BITamin/project_data/resnet_model_cnn.pt'
    Resnet = torch.load(PATH)
    pred1 = Resnet(x)
    pred1 = pred1.detach().numpy()
    '''
    if pred1 < 0:
        pred1 = 0.0
    
    elif pred1 > 1:
        pred1 = 1.0
    '''
    
    return pred1

In [78]:
def model2(x):    
    delisting = pd.read_csv('/Users/parkjeonghoon/Desktop/BITamin/project_data/ratios/delisting_ratios.csv', dtype={'stock_code': str})
    listing = pd.read_csv('/Users/parkjeonghoon/Desktop/BITamin/project_data/ratios/not_delisting_ratios.csv', dtype={'stock_code': str})

    delisting.drop('Unnamed: 0',axis = 1,inplace=True)
    listing.drop('Unnamed: 0' ,axis = 1, inplace=True)

    #0,1라벨링
    listing['label'] = 0
    delisting['label'] = 1

    delisting = delisting.fillna(0)
    df =pd.concat([delisting, listing])
    
    train_columns = df.columns.difference(['stock_code','label'])
    x_train, x_test, y_train, y_test = train_test_split(df[train_columns],df['label'],test_size=0.2, random_state=42)
    model = LogisticRegression(random_state=42)
    model.fit(x_train, y_train)

    pred2 = model.predict(x)
    
    return pred2

In [79]:
def tot_model(x1, x2):
    w1, w2 = 0.5, 0.5
    
    p1 = model1(x1)
    p2 = model2(x2)
    
    result = w1*p1 + w2*p2
    return p1, p2, result

In [57]:
test_delisting = pd.read_csv('...', dtype={'stock_code': str})
test_listing = pd.read_csv('...', dtype={'stock_code': str})

test_delisting.drop('Unnamed: 0',axis = 1,inplace=True)
test_listing.drop('Unnamed: 0' ,axis = 1, inplace=True)

FileNotFoundError: [Errno 2] No such file or directory: '...'

In [81]:
class CustomDataset(Dataset):
    def __init__(self, data_list, transform=None):
        self.data_list = data_list

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        data = self.data_list[idx]
        graph = data['graph']
        label = data['label']
        code = data['code']

        return code, graph, label

In [82]:
dic_path = '/Users/parkjeonghoon/Desktop/BITamin/project_data/model_test_pt'
test_list = os.listdir(dic_path)
test_list = [filename.split('.')[0] for filename in test_list]

# 데이터 파일들의 경로 리스트 (각 회사별 데이터 경로)
test_files = []

for code_t in test_list:
    test_files.append(f'{code_t}')

# 데이터를 저장할 리스트
test_dataset = []

delisting = pd.read_csv('/Users/parkjeonghoon/Desktop/BITamin/project_data/test_fs/delisting_ratio_test.csv', dtype={'stock_code': str})
# listing = pd.read_csv('/Users/parkjeonghoon/Desktop/BITamin/project_data/ratios/not_delisting_ratios.csv', dtype={'stock_code': str})

delisting.drop('Unnamed: 0',axis = 1,inplace=True)
# listing.drop('Unnamed: 0' ,axis = 1, inplace=True)

#0,1라벨링
# listing['label'] = 0
# delisting['label'] = 1

delisting = delisting.fillna(0)
# df =pd.concat([delisting, listing])

# 각 데이터 파일을 로드하여 데이터셋에 추가
for test_code in test_files:
    file_test_path = f'/Users/parkjeonghoon/Desktop/BITamin/project_data/model_test_pt/{test_code}.pt'
    print(test_code)
    print(file_test_path)
    test = torch.load(file_test_path)  # .pt 파일 로드
    test['code'] = f'{test_code}'
    test_dataset.append(test)

print(len(test_files))
# CustomDataset 클래스를 사용하여 데이터셋 생성
test_set = CustomDataset(test_dataset)

# 데이터로더 생성
batch_size = 1
test_loader = DataLoader(test_set, batch_size= batch_size, shuffle=False)

251960
/Users/parkjeonghoon/Desktop/BITamin/project_data/model_test_pt/251960.pt
271780
/Users/parkjeonghoon/Desktop/BITamin/project_data/model_test_pt/271780.pt
204990
/Users/parkjeonghoon/Desktop/BITamin/project_data/model_test_pt/204990.pt
152330
/Users/parkjeonghoon/Desktop/BITamin/project_data/model_test_pt/152330.pt
4


In [83]:
for batch, data in enumerate(test_loader):
    code, graph, label = data
    df = delisting[delisting['stock_code'].isin([code][0])]
    data_f = df.columns.difference(['stock_code','label'])
    graph = Variable(graph.view(1, 3, 480, 640))
    label = Variable(label)
    p1, p2, result = tot_model(graph, df[data_f])
    print(p1, p2, result)

[[-0.69537526]] [0] [[-0.34768763]]
[[-0.7295942]] [0] [[-0.36479709]]
[[-0.67081994]] [1] [[0.16459003]]
[[-0.7877577]] [0] [[-0.39387885]]
